In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import random
import string
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from collections import Counter

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import balanced_accuracy_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils.class_weight import compute_sample_weight

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path_BTC = "/content/drive/MyDrive/BTCUSD_PERP-1m-2023-07-04.csv"
path_ETH = "/content/drive/MyDrive/ETHUSD_PERP-1m-2023-07-04.csv"
BTC = pd.read_csv(path_BTC)
ETH = pd.read_csv(path_ETH)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
'''
test_path = "/content/drive/MyDrive/BTCUSDT-1s-2024-02-24.csv"
test = pd.read_csv(test_path)
print(test.head())
'''

'\ntest_path = "/content/drive/MyDrive/BTCUSDT-1s-2024-02-24.csv"\ntest = pd.read_csv(test_path)\nprint(test.head())\n'

In [ ]:
BTC.head()

,open_time,open,high,low,close,volume,close_time,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,ignore
0,1688428800000,31152.8,31169.9,31152.7,31159.5,25756,1688428859999,82.645778,454,18659,59.875829,0
1,1688428860000,31159.6,31161.8,31159.5,31161.8,11290,1688428919999,36.231926,107,10238,32.855982,0
2,1688428920000,31161.8,31161.8,31152.0,31152.0,26219,1688428979999,84.156203,360,10491,33.676462,0
3,1688428980000,31152.0,31152.0,31126.7,31126.7,22268,1688429039999,71.499067,451,1976,6.344558,0
4,1688429040000,31126.7,31142.3,31126.7,31142.2,12839,1688429099999,41.243000,247,12446,39.980745,0


In [ ]:
ETH.head()

,open_time,open,high,low,close,volume,close_time,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,ignore
0,1688428800000,1954.41,1954.41,1953.83,1953.84,35373,1688428859999,181.014185,209,5817,29.769889,0
1,1688428860000,1953.84,1953.84,1953.52,1953.52,32485,1688428919999,166.266342,137,15331,78.465995,0
2,1688428920000,1953.52,1953.52,1952.83,1952.83,13876,1688428979999,71.046156,155,988,5.058884,0
3,1688428980000,1952.83,1952.83,1951.41,1951.41,59953,1688429039999,307.133894,268,478,2.449171,0
4,1688429040000,1951.40,1953.56,1951.40,1953.56,63308,1688429099999,324.248311,301,62603,320.636995,0


In [ ]:
df_btc_prefixed = BTC.add_prefix('BTC_')
df_eth_prefixed = ETH.add_prefix('ETH_')

# Concatenate the two DataFrames
combined_df = pd.concat([df_btc_prefixed, df_eth_prefixed], axis=1)

combined_df.rename(columns={'BTC_open_time': 'open_time'}, inplace=True)
combined_df.rename(columns={'BTC_close_time': 'close_time'}, inplace=True)
combined_df = combined_df.drop(columns=['ETH_open_time', 'ETH_close_time'])

combined_df = combined_df.reindex(sorted(combined_df.columns, reverse = True), axis=1)
pd.set_option('display.max_columns', None)  # Set to None to display all columns
print(combined_df.head())
pd.reset_option('display.max_columns')

       open_time     close_time  ETH_volume  ETH_taker_buy_volume  \
0  1688428800000  1688428859999       35373                  5817   
1  1688428860000  1688428919999       32485                 15331   
2  1688428920000  1688428979999       13876                   988   
3  1688428980000  1688429039999       59953                   478   
4  1688429040000  1688429099999       63308                 62603   

   ETH_taker_buy_quote_volume  ETH_quote_volume  ETH_open  ETH_low  \
0                   29.769889        181.014185   1954.41  1953.83   
1                   78.465995        166.266342   1953.84  1953.52   
2                    5.058884         71.046156   1953.52  1952.83   
3                    2.449171        307.133894   1952.83  1951.41   
4                  320.636995        324.248311   1951.40  1951.40   

   ETH_ignore  ETH_high  ETH_count  ETH_close  BTC_volume  \
0           0   1954.41        209    1953.84       25756   
1           0   1953.84        137    1953